In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
# Need to normalize all the user's ratings before finding cosine_similarity
# - possibly using sklearn.preprocessing.normalize
# - need to deal with nans and/or zero

import numpy as np
import pandas as pd

df = pd.read_csv('inputs/boardgame-elite-users.csv')
df = df.pivot_table(index='userID', columns='gameID', values='rating')



df.fillna(0, inplace=True)

print(df.head(1))

gameID    3    5   10   11   12   13   18   41   42   45 ...   194655  198773  \
userID                                                   ...                    
272     1.0  1.0  1.0  0.0  1.0  7.5  1.0  4.0  1.0  6.0 ...      0.0     0.0   

gameID  201808  204583  205059  205637  209010      mean   max  min  
userID                                                               
272        0.0     0.0     0.0     0.0     0.0  1.611722  10.0  1.0  

[1 rows x 405 columns]


In [39]:
def norm(someRow):
    return someRow['rating'] - someRow['min'] / someRow['max'] - someRow['min']

print(df > 0])

# df[df['rating'] is not null] = df.apply(norm)
# from sklearn.metrics.pairwise import pairwise_distances
# user_similarity = pairwise_distances(np.array(df), metric='cosine')

KeyError: '[(1.0, 1.0, 1.0, 0.0, 1.0, 7.5, 1.0, 4.0, 1.0, 6.0, 1.0, 1.0, 0.0, 3.0, 1.0, 1.0, 0.0, 1.0, 1.0, 5.0, 2.0, 1.0, 1.0, 6.0, 1.0, 5.0, 1.0, 9.0, 1.0, 1.0, 0.0, 9.0, 1.0, 8.0, 1.0, 10.0, 0.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 9.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 3.0, 1.0, 0.0, 0.0, 0.0, 1.5, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 3.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 6.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 5.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 5.0, 1.0, 0.0, 1.0, 1.0, 6.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 6.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1